In [2]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [3]:
sys.path.append('../..')

In [4]:
from lung_cancer.dataset import FilesIndex, Dataset, Pipeline
from lung_cancer import CTImagesMaskedBatch as CTIMB

Using TensorFlow backend.


### Build histo of nodules' positions

* Dataset: Luna, unified spacing

In [5]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
lunaix = FilesIndex(path=DIR_LUNA, no_ext=True)
lunaset = Dataset(index=lunaix, batch_class=CTIMB)

In [6]:
len(lunaset.indices)

888

* Nodules csv-database

In [7]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')

* Random histo

In [8]:
# a bit of hardcode
SHAPE = [384, 448, 448]
ranges = list(zip([0]*3, SHAPE))
histo = list(np.histogramdd(np.random.uniform(low=0, high=1, size=(100, 3)) *
                            np.array(SHAPE).reshape(1, -1), range=ranges, bins=4))

In [9]:
histo[0].sum()

100.0

### Use the histo to sample nodules

In [63]:
NOD_SHAPE = (32, 64, 64)

pipe = (Pipeline()
        .load(fmt='raw')
        .fetch_nodules_info(nodules)
        .unify_spacing(shape=SHAPE, spacing=(1.7, 1.0, 1.0))
        .create_mask()
        .sample_nodules(batch_size=10, nodule_size=NOD_SHAPE, share=0.5,
                        histo=histo, variance=(20, 70, 70))
       )

In [64]:
gen = (lunaset >> pipe).gen_batch(batch_size=5, n_epochs=None)

In [65]:
nods = next(gen)

### Plots

In [66]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [67]:
nods.images.shape

(320, 64, 64)

In [68]:
nodnum = 4

In [69]:
nodix = nods.indices[nodnum]

In [70]:
interact(lambda height: plot_arr_slices(height, 
                                        nods.get(nodix, 'images'),
                                        nods.get(nodix, 'masks'),
                                        nods.get(nodix, 'masks')),
         height=(0.01, 0.99, 0.01))

A Jupyter Widget

<function __main__.<lambda>>

### Get only (and all) cancerous nodules

In [128]:
bch = CTIMB(lunaix.create_subset(lunaix.indices[[100, 110, 120, 130, 140]]))

In [129]:
bch = bch.load(fmt='raw')

In [130]:
bch = bch.fetch_nodules_info(nodules)
bch = bch.create_mask()
bch = bch.unify_spacing(shape=SHAPE, spacing=[1.7, 0.9, 0.9])

In [131]:
only_cancer = bch.sample_nodules(nodule_size=NOD_SHAPE, batch_size=None, share=0.6,
                                 variance=(49, 196, 196))

In [132]:
len(only_cancer)

15

In [134]:
only_cancer.num_nodules

10

In [146]:
nodnum = 2

In [147]:
nodix = only_cancer.indices[nodnum]

In [148]:
interact(lambda height: plot_arr_slices(height, 
                                        only_cancer.get(nodix, 'images'),
                                        only_cancer.get(nodix, 'masks'),
                                        only_cancer.get(nodix, 'masks')),
         height=(0.01, 0.99, 0.01))

A Jupyter Widget

<function __main__.<lambda>>